# Env Set Up #

In [2]:
# Import libraries
import os
import cv2
import time
import random
import gc as G
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras import backend as K, mixed_precision
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import keras_tuner as kt
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau as RLOP, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier as KC
from sklearn.model_selection import RandomizedSearchCV as RSCV
from utils import import_train, import_others, view_train_images, plot_training_results
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization



In [3]:
# Set GPU memory limit (3GB out of 4GB)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3072)]  # 3GB = 3072MB
        )
    except RuntimeError as e:
        print(e)

# 1) Data Import #

In [4]:
# paths for data
train_dir = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Data/CLS-LOC/train"
val_dir = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Data/CLS-LOC/val"
test_dir = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Data/CLS-LOC/test"

# path to save tuner results
kt_path = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/tuning_res/kt"


In [5]:
# data import
train_images, train_labels = import_train(train_dir, 50000)
test_images, test_labels = import_train(val_dir, 10000)

Images processed: 12500 (25%)
Images processed: 25000 (50%)
Images processed: 37500 (75%)
Function processed C:/Users/RoiMinuit/Desktop/data/ILSVRC/Data/CLS-LOC/train in 283 seconds.

Images processed: 2500 (25%)
Images processed: 5000 (50%)
Images processed: 7500 (75%)
Function processed C:/Users/RoiMinuit/Desktop/data/ILSVRC/Data/CLS-LOC/val in 92 seconds.



In [6]:
# prepare training data
X_train = np.array(train_images)
X_train = np.repeat(X_train, 3, -1)

Y_train = [str(s) for s in train_labels]
label_encoder = LabelEncoder()
Y_train = label_encoder.fit_transform(Y_train)
Y_hot = to_categorical(Y_train, num_classes=1000)


In [7]:
# prepare testing data
X_test = np.array(test_images)
X_test = np.repeat(X_test, 3, -1)

Y_test = [str(s) for s in test_labels]
label_encoder = LabelEncoder()
Y_test = label_encoder.fit_transform(Y_test)
Y_test_hot = to_categorical(Y_test, num_classes=1000)


In [8]:
# check data shape
print(Y_train.shape)
print(Y_test.shape)

print(len(np.unique(Y_train)))
print(len(np.unique(Y_test)))

(50000,)
(10000,)
1000
1


# 2) Model Optimization #

## a) Keras Classifier ##

In [70]:
# parameters for tuning
epx = 50
batch_size = 8
RSCV_opts = ['adam', 'rmsprop']
krnl = (3, 3)
in_shape = (64, 64, 3)
RSCV_activation = ['relu', 'tanh', 'sigmoid']
RSCV_node1 = [2, 4, 8, 16, 32]
RSCV_node2 = [2, 4, 8, 16, 32]
RSCV_drop = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
RSCV_bacth = [2, 4, 6, 8, 16, 32, 64, 128]

params_grid = dict(node1 = RSCV_node1,
               node2 = RSCV_node2,
               activations = RSCV_activation,
               optimizers = RSCV_opts,
               batch_size = RSCV_bacth,
               drops = RSCV_drop)

In [90]:
# Keras Classifier needs a model builder function to instantiate a new model for each new grid point
def build_CNN(optimizers, node1, activations, node2, drops):
    CNN = Sequential([
        # first layer
        Conv2D(node1, kernel_size=krnl, activation=activations, input_shape=in_shape),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        # 2nd layer
        Conv2D(node2, kernel_size=krnl, activation=activations),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        # output layer
        GlobalAveragePooling2D(),
        Dense(drops),
        Dense(1000, 'softmax')
    ])
    CNN.compile(optimizers,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    # return built model
    return CNN

In [91]:
# build Keras Classifier
RS_cnn = KC(build_fn=build_CNN,
            verbose=1,
            epochs=5,
            batch_size=batch_size)

C:\Users\RoiMinuit\AppData\Local\Temp\ipykernel_41516\1010408690.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  RS_cnn = KC(build_fn=build_CNN,


In [ ]:
# establish search space and search
grid_search = RSCV(RS_cnn,
                   param_distributions=params_grid,
                   cv=5,
                   scoring='accuracy',
                   verbose=1)

# execute search with categorical Y, not one-hotted
grid_search.fit(X_train, Y_train)

## b) Keras Tuner ##

In [9]:
# variables for tuning
units = [2, 4, 6, 8, 10, 12, 14, 16, 32, 64, 128]
activations = ['relu', 'linear', 'sigmoid', 'selu', 'elu']
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('ImgNet_Model.h5', monitor='val_loss', save_best_only=True, save_freq='epoch')

In [15]:
# base model for kt
def build_KT_CNN(hp):
    with tf.device('/GPU:0'):
        layers = [
            # input layer
            Conv2D(hp.Int('filters', min_value=2, max_value=128),
                input_shape=(64, 64, 3),
                kernel_size=(3, 3),
                activation=hp.Choice('activation', activations)),
            BatchNormalization(),
            MaxPooling2D(2, 2),
            Dropout(hp.Float('drop_rate', 1e-4, 1e-1))
        ]
        
        # hidden layer(s)
        for l in range(hp.Int('num_layers', 1, 5)):
            layers.append(Conv2D(hp.Int('filters', min_value=2, max_value=128),
                                kernel_size=(3, 3),
                                activation=hp.Choice('activation', activations)))
            layers.append(BatchNormalization())
            layers.append(MaxPooling2D(2, 2))
            layers.append(Dropout(hp.Float('drop_rate', 1e-4, 1e-1)))
        
        # output layer
        layers.append(GlobalAveragePooling2D())
        layers.append(Dropout(hp.Float('drop_rate', 1e-4, 1e-1)))
        layers.append(Dense(1000, activation=hp.Choice('activation', activations)))
        
        CNN = Sequential(layers)
        lr = hp.Float('learning_rate', min_value=1e-5, max_value=1e-1, sampling='LOG', default=1e-3)
        CNN.compile(optimizer=Adam(learning_rate=lr),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
    # return built model
    return CNN

build_KT_CNN(kt.HyperParameters())


In [ ]:
# eastablish search space and search
tuner = kt.RandomSearch(
    hypermodel=build_KT_CNN,
    objective='val_accuracy',
    max_trials=2,
    executions_per_trial=2,
    overwrite=True,
    directory=kt_path,
    project_name='ImgNetOptimization'
)

# search
tuner.search(X_train, Y_train, epochs=2, validation_data=(X_test, Y_test), callbacks=[early_stop, checkpoint])


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
26                |26                |filters
linear            |linear            |activation
0.025642          |0.025642          |drop_rate
2                 |2                 |num_layers
0.00043102        |0.00043102        |learning_rate

Epoch 1/2


In [13]:
# view optimized model
model = tuner.get_best_models()[0]
model.summary()

ValueError: Exception encountered when calling layer "conv2d_4" (type Conv2D).

Negative dimension size caused by subtracting 3 from 2 for '{{node conv2d_4/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, conv2d_4/Conv2D/ReadVariableOp)' with input shapes: [?,2,2,120], [3,3,120,120].

Call arguments received by layer "conv2d_4" (type Conv2D):
  • inputs=tf.Tensor(shape=(None, 2, 2, 120), dtype=float32)

In [50]:
tuner.results_summary()

Results summary
Results in C:/Users/RoiMinuit/Desktop/data/ILSVRC/tuning_res/kt\ImgNetOptimization
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
units: 56
activation: relu
num_layers: 3
nodes: 34
drop_rate: 0.056074816973524635
learning_rate: 0.003601625947816291
Score: 1.0

Trial 4 summary
Hyperparameters:
units: 6
activation: elu
num_layers: 2
nodes: 103
drop_rate: 0.034321311243740625
learning_rate: 0.0009050440273521296
Score: 0.00041666667675599456

Trial 3 summary
Hyperparameters:
units: 114
activation: linear
num_layers: 1
nodes: 61
drop_rate: 0.027326859907895323
learning_rate: 0.05328040337572006
Score: 0.0

Trial 1 summary
Hyperparameters:
units: 97
activation: elu
num_layers: 5
nodes: 100
drop_rate: 0.09236573698403451
learning_rate: 2.1778293990354973e-05
Traceback (most recent call last):
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and